### IMPORTS

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

### ORGANIZANDO ARQUIVOS E CARREGANDO DADOS

In [ ]:
BASE_DIR = Path.cwd().parent
RAW_DIR = BASE_DIR / "data" / "raw"
PROC_DIR = BASE_DIR / "data" / "processed"

PRODUCAO_PATH = RAW_DIR / "producao_mensal.xlsx"
CONSOL_PATH = PROC_DIR / "financials_consolidated.xlsx"

df_prod = pd.read_excel(PRODUCAO_PATH, parse_dates=["data"])
df_agg = pd.read_excel(CONSOL_PATH, parse_dates=["data"])

# Conferir colunas
print("Colunas producao:", df_prod.columns.tolist())
print("Colunas agregados:", df_agg.columns.tolist())


### LIMPEZA RAPIDA / TIPOS E ORDENAÇÃO

In [ ]:
# Garantir ordenação e tipos corretos
df_prod = df_prod.sort_values(["nome_campo", "data"]).reset_index(drop=True)
df_agg = df_agg.sort_values("data").reset_index(drop=True)

# Converter colunas numéricas (se houver problemas de leitura)
numeric_cols_prod = ["producao_barris", "receita", "custo_operacional", "custo_geral_brl", "lucro_liquido_brl"]
for c in numeric_cols_prod:
    if c in df_prod.columns:
        df_prod[c] = pd.to_numeric(df_prod[c], errors="coerce")

numeric_cols_agg = ["producao_total_barris", "receita_total_brl", "custo_operacional_total_brl", "custo_geral_total_brl", "lucro_total_brl"]
for c in numeric_cols_agg:
    if c in df_agg.columns:
        df_agg[c] = pd.to_numeric(df_agg[c], errors="coerce")

print("Registros producao:", len(df_prod))
print("Registros agregados:", len(df_agg))


### PRODUÇÃO TOTAL, RECEITA E LUCRO : (PLOT) SERIES TEMPORAIS

In [ ]:
# Séries temporais agregadas: produção, receita, lucro
fig, ax = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

ax[0].plot(df_agg["data"], df_agg["producao_total_barris"] / 1e6)  # em milhões de barris
ax[0].set_ylabel("Produção (10^6 barris)")
ax[0].set_title("Produção Total Mensal")

ax[1].plot(df_agg["data"], df_agg["receita_total_brl"] / 1e6)
ax[1].set_ylabel("Receita (milhões BRL)")
ax[1].set_title("Receita Total Mensal")

ax[2].plot(df_agg["data"], df_agg["lucro_total_brl"] / 1e6)
ax[2].set_ylabel("Lucro (milhões BRL)")
ax[2].set_title("Lucro Total Mensal")
ax[2].set_xlabel("Data")

plt.tight_layout()
plt.show()


### MARCAÇÃO DE EVENTOS IMPORTANTES COM INTERVALOD E 2 ANOS (2008, 2014, 2020, 2022)

In [ ]:
# Função utilitraia para desenhar linhas de eventos nos plot
def marcar_eventos(ax, years=[2008, 2014, 2020, 2022], ymin=None, ymax=None, label_offset=0.02):
    for y in years:
        date = pd.Timestamp(year=y, month=1, day=1)
        ax.axvline(date, color="red", linestyle="--", linewidth=1)
        if label_offset is not None:
            ax.text(date, ymax - label_offset*(ymax-ymin), str(y), color="red", rotation=90, va="top", ha="right")

# Exemplo aplicado em produção
fig, ax = plt.subplots(1, 1, figsize=(14,4))
ax.plot(df_agg["data"], df_agg["producao_total_barris"] / 1e6)
ax.set_ylabel("Produção (10^6 barris)")
ax.set_title("Produção Total Mensal — com eventos")
marcar_eventos(ax, years=[2008,2014,2020,2022], ymin=df_agg["producao_total_barris"].min(), ymax=df_agg["producao_total_barris"].max())
plt.show()


### DECOMPOSIÇÃO SAZONAL (STATSMODEL) DA SÉRIE DE PRODUÇÃO TOTAL

In [ ]:
s = df_agg.set_index("data")["producao_total_barris"].asfreq("MS")  # frequência mensal
decomp = seasonal_decompose(s, model="additive", period=12, extrapolate_trend="freq")

fig = decomp.plot()
fig.set_size_inches(12,9)
plt.suptitle("Decomposição Sazonal - Produção Total (aditiva)", fontsize=14)
plt.tight_layout()
plt.show()


### DECOMPOSIÇÃO PRA RECEITA E LUCRO

In [ ]:
for col in ["receita_total_brl", "lucro_total_brl"]:
    s = df_agg.set_index("data")[col].asfreq("MS")
    decomp = seasonal_decompose(s, model="additive", period=12, extrapolate_trend="freq")
    fig = decomp.plot()
    fig.set_size_inches(12,9)
    plt.suptitle(f"Decomposição Sazonal - {col}", fontsize=14)
    plt.tight_layout()
    plt.show()


### DETECT PEAKS/DROPS IDENTIFICAR RUPTURAS/GRANDES MUDANCAS(SIMPLES)

In [ ]:
# Diferença percentual mês a mês para receita e produção
df_agg["pct_change_producao"] = df_agg["producao_total_barris"].pct_change()
df_agg["pct_change_receita"] = df_agg["receita_total_brl"].pct_change()

# Exibir meses com maior variação negativa e positiva
print("Top quedas (produção):")
display(df_agg.nsmallest(5, "pct_change_producao")[["data","producao_total_barris","pct_change_producao"]])

print("Top picos (produção):")
display(df_agg.nlargest(5, "pct_change_producao")[["data","producao_total_barris","pct_change_producao"]])


### FEATURES POTENCIAIS PARA ML (LAGS, ROLLING MEANS, CHOQUE)

In [ ]:
df_features = df_agg.copy().set_index("data")

# Lags do preço — precisamos recuperar preço médio USD ou BRL por mês; se não existir, podemos derivar preço médio como receita/produção
df_features["preco_medio_brl"] = df_features["receita_total_brl"] / df_features["producao_total_barris"]
df_features["preco_medio_brl"].replace([np.inf, -np.inf], np.nan, inplace=True)

# Lags
for lag in [1,3,6]:
    df_features[f"preco_lag_{lag}"] = df_features["preco_medio_brl"].shift(lag)
    df_features[f"producao_lag_{lag}"] = df_features["producao_total_barris"].shift(lag)

# Rolling means
df_features["producao_roll_3"] = df_features["producao_total_barris"].rolling(3).mean()
df_features["producao_roll_12"] = df_features["producao_total_barris"].rolling(12).mean()

# Choques indexados (indicadores binários por eventos)
for y in [2008,2014,2020,2022]:
    df_features[f"shock_{y}"] = ((df_features.index.year == y)).astype(int)

# Limpar e mostrar
display(df_features.head(14))


### SALVAR DATASET PARA ML

In [ ]:
OUT_PATH = PROC_DIR / "ml_dataset_features.csv"
df_features.reset_index().to_csv(OUT_PATH, index=False)
print("Dataset de features salvo em:", OUT_PATH)


### CORRELAÇÃO E CHECAGEM RAPIDA

In [ ]:
# Correlação entre preço médio e receita, produção
corr = df_features[["preco_medio_brl","producao_total_barris","receita_total_brl","lucro_total_brl"]].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu_r")
plt.title("Correlação entre variáveis-chave")
plt.show()
